In [ ]:
import deepchem

from deepchem.utils.save import load_from_disk
dataset_file= "gcn_ratio.csv"

loader = deepchem.data.CSVLoader(tasks=["rat"], 
                                 smiles_field="smiles", 
                                 featurizer=deepchem.feat.ConvMolFeaturizer())
# Featurizing the dataset with ConvMolFeaturizer
dataset = loader.featurize(dataset_file)


In [ ]:
len(dataset)

In [ ]:
splitter = deepchem.splits.RandomSplitter()
train, valid, _ = splitter.train_valid_test_split(dataset,
                                                  frac_train=0.8,
                                                  frac_valid=0.1,
                                                  frac_test=0.1)
normalizer = deepchem.trans.NormalizationTransformer(transform_y=True, 
                                                     dataset=train, 
                                                     move_mean=True)
train = normalizer.transform(train)
test = normalizer.transform(valid)

In [ ]:
from deepchem.models import GraphConvModel
graph_conv = GraphConvModel(1,
                            batch_size=256,
                            mode="regression")
metric = deepchem.metrics.Metric(deepchem.metrics.pearson_r2_score)

In [ ]:
# Fitting the model
graph_conv.fit(train, nb_epoch=40)

In [ ]:
valid_scores = graph_conv.evaluate(valid, [metric], [normalizer])
print(f"Validation Scores: {valid_scores}")